In [53]:
import pandas as pd
import wbgapi as wb
import numpy as np

In [54]:
df = wb.data.DataFrame(['NY.GNP.PCAP.CD','SI.POV.GINI'], time=range(2010,2023), skipBlanks =True, columns='series')

In [55]:
df

NY.GNP.PCAP.CD  SI.POV.GINI
economy time                               
ABW     YR2010         23390.0          NaN
        YR2011         23400.0          NaN
        YR2012         24440.0          NaN
        YR2013         25500.0          NaN
        YR2014         25790.0          NaN
...                        ...          ...
ZWE     YR2017          1170.0         44.3
        YR2018          1550.0          NaN
        YR2019          1450.0         50.3
        YR2020          1460.0          NaN
        YR2021          1530.0          NaN

[2917 rows x 2 columns]

In [56]:
df = df.reset_index()

In [57]:
df

,economy,time,NY.GNP.PCAP.CD,SI.POV.GINI
0,ABW,YR2010,23390.0,NaN
1,ABW,YR2011,23400.0,NaN
2,ABW,YR2012,24440.0,NaN
3,ABW,YR2013,25500.0,NaN
4,ABW,YR2014,25790.0,NaN
...,...,...,...,...
2912,ZWE,YR2017,1170.0,44.3
2913,ZWE,YR2018,1550.0,NaN
2914,ZWE,YR2019,1450.0,50.3
2915,ZWE,YR2020,1460.0,NaN


In [58]:
df = df.rename(columns={'economy':'iso3', 'time':'year', 'NY.GNP.PCAP.CD':'gnipc', 'SI.POV.GINI':'gini'})
df['year'] = df['year'].str[2:]
df['year'] = df['year'].astype('int')
df

,iso3,year,gnipc,gini
0,ABW,2010,23390.0,NaN
1,ABW,2011,23400.0,NaN
2,ABW,2012,24440.0,NaN
3,ABW,2013,25500.0,NaN
4,ABW,2014,25790.0,NaN
...,...,...,...,...
2912,ZWE,2017,1170.0,44.3
2913,ZWE,2018,1550.0,NaN
2914,ZWE,2019,1450.0,50.3
2915,ZWE,2020,1460.0,NaN


In [59]:
#df = df.pivot(index='economy', columns='time', values='NY.GNP.PCAP.CD')

In [60]:
#df = pd.melt(df, id_vars = 'iso3', var_name='year', value_name='gnipc')

In [61]:
df

,iso3,year,gnipc,gini
0,ABW,2010,23390.0,NaN
1,ABW,2011,23400.0,NaN
2,ABW,2012,24440.0,NaN
3,ABW,2013,25500.0,NaN
4,ABW,2014,25790.0,NaN
...,...,...,...,...
2912,ZWE,2017,1170.0,44.3
2913,ZWE,2018,1550.0,NaN
2914,ZWE,2019,1450.0,50.3
2915,ZWE,2020,1460.0,NaN


In [62]:
df

,iso3,year,gnipc,gini
0,ABW,2010,23390.0,NaN
1,ABW,2011,23400.0,NaN
2,ABW,2012,24440.0,NaN
3,ABW,2013,25500.0,NaN
4,ABW,2014,25790.0,NaN
...,...,...,...,...
2912,ZWE,2017,1170.0,44.3
2913,ZWE,2018,1550.0,NaN
2914,ZWE,2019,1450.0,50.3
2915,ZWE,2020,1460.0,NaN


In [63]:
df.isna().sum()

iso3        0
year        0
gnipc       3
gini     2099
dtype: int64

In [64]:
df.dtypes

iso3      object
year       int64
gnipc    float64
gini     float64
dtype: object

In [66]:
df.describe()

,year,gnipc,gini
count,2917.000000,2914.000000,818.000000
mean,2015.483716,13820.009869,36.288142
std,3.431801,19066.244197,7.495885
min,2010.000000,210.000000,23.200000
25%,2013.000000,1949.402948,30.525000
50%,2015.000000,5795.000000,35.200000
75%,2018.000000,15700.000000,41.100000
max,2021.000000,122470.000000,63.400000


In [75]:
df.gini.dropna().value_counts()

33.2    9
30.8    9
26.8    9
27.2    8
38.8    8
       ..
54.0    1
59.1    1
46.2    1
51.6    1
50.3    1
Name: gini, Length: 264, dtype: int64

In [74]:
df.gini.dropna().nunique()

264